In [1]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import altair as alt
alt.data_transformers.disable_max_rows()
import sys
sys.path.append("..")
from network_analysis.load_datasets import get_updated_shxco_data

from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import GridSearchCV
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel

from scipy import spatial, stats
# from scipy.stats import pearsonr

members_df, books_df, borrow_events, events_df = get_updated_shxco_data(get_subscription=False)


In [2]:
grouped_borrows = borrow_events[['member_id', 'item_uri']].groupby(['member_id', 'item_uri']).size().reset_index(name='counts')
pivoted_borrows = grouped_borrows.pivot(
    index='member_id', columns='item_uri', values='counts').fillna(0).astype(int)
df = pivoted_borrows.copy()

In [3]:
scaler = StandardScaler()
weighted = scaler.fit_transform(df)
weighted = pd.DataFrame(weighted, index=df.index, columns=df.columns)



In [4]:
def get_formatted_table_titles(sorted_values, numb_of_preds):
  titles = []
  for i in sorted_values[0:numb_of_preds].item_uri.tolist():
    item = books_df[books_df.id == i]
    if item.author.isna().any() == False:
      author = ' '.join(item.author.str.split(',').values[0][::-1])
      # author = ', ' + author
    else: 
      author = '(Periodical)'
    title = item.title.values[0]
    titles.append(f"*{title}*,<br>{author}")
  return titles

def get_formatted_chart_titles(sorted_values, numb_of_preds):
  titles = []
  for i in sorted_values[0:numb_of_preds].item_uri.tolist():
    item = books_df[books_df.id == i]
    if item.author.isna().any() == False:
      author = ' '.join(item.author.str.split(',').values[0][::-1])
      author = 'by' + author
    else: 
      author = '(Periodical)'
    title = item.title.values[0] 
    titles.append(f"{title} {author}")
  return titles

In [5]:
partial_df = pd.read_csv('../dataset_generator/data/partial_borrowers_collapsed.csv')
partial_df[0:1]
partial_members = ['hemingway-ernest']
query_members = grouped_borrows.loc[grouped_borrows.member_id.isin(partial_members)].member_id.unique().tolist()
partial_df = partial_df[partial_df.member_id.isin(query_members)]


In [6]:
d = (grouped_borrows.groupby(['member_id'])['item_uri', 'counts'].apply(
     lambda x: dict(x.values)).to_dict())

In [7]:
# Returns a distance-based similarity score for person1 and person2
def euc_distance(prefs,person1,person2):
    si = {} 
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1    
    if len(si) == 0: 
        return 0
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item],2) 
                          for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)


def get_predictions(person, prefs, weighted, query_books):

    totals = {} 
    simSums = {}

    cos_totals = {} 
    cos_simSums = {}

    pear_totals = {} 
    pear_simSums = {}
    for other in prefs:

        if other == person:
            continue
        member_vector = weighted.loc[person]
        other_vector = weighted.loc[other]

        euc_vector = euc_distance(prefs, person, other)
        cosine_score = spatial.distance.cosine(member_vector,other_vector)
        pearson_score, pearson_p = stats.pearsonr(member_vector, other_vector)
        if euc_vector <= 0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item] == 0:
                if item in query_books:

                    totals.setdefault(item,0) 
                    totals[item] += prefs[other][item] * euc_vector
                    simSums.setdefault(item,0)
                    simSums[item] += euc_vector

                    cos_totals.setdefault(item,0) 
                    cos_totals[item] += prefs[other][item] * cosine_score
                    cos_simSums.setdefault(item,0)
                    cos_simSums[item] += cosine_score

                    pear_totals.setdefault(item,0) 
                    pear_totals[item] += prefs[other][item] * pearson_score
                    pear_simSums.setdefault(item,0)
                    pear_simSums[item] += pearson_score
                    pear_simSums.setdefault(item,0)
                    pear_simSums[item] += pearson_score
    rankings = [(total/simSums[item], item) for item, total in totals.items()]
    rankings.sort()
    rankings.reverse()

    cos_rankings = [(total/cos_simSums[item], item) for item, total in cos_totals.items()]
    cos_rankings.sort()
    cos_rankings.reverse()

    pear_rankings = [(total/pear_simSums[item], item) for item, total in pear_totals.items()]
    pear_rankings.sort()
    pear_rankings.reverse()

    return rankings, cos_rankings, pear_rankings

In [8]:
partial_df['subscription_starttime'] = pd.to_datetime(partial_df['subscription_start'])
partial_df['subscription_endtime'] = pd.to_datetime(partial_df['subscription_end'])

In [9]:

# load members, books, events as csv
events_df = pd.read_csv('../dataset_generator/source_data/SCoData_events_v1.2_2022-01.csv')
# split multiple members for shared accounts in events
events_df[
    ["first_member_uri", "second_member_uri"]
] = events_df.member_uris.str.split(";", expand=True)

# working with the first member for now...
# generate short ids equivalent to those in member and book dataframes
events_df["member_id"] = events_df.first_member_uri.apply(
    lambda x: x.split("/")[-2]
)
events_df["item_uri"] = events_df.item_uri.apply(
    lambda x: x.split("/")[-2] if pd.notna(x) else None
)

events_df['start_datetime'] = pd.to_datetime(events_df['start_date'], errors='coerce')
events_df['end_datetime'] = pd.to_datetime(events_df['end_date'], errors='coerce')

In [192]:
n = 200
final_df = []
previous_books = None
narrow_query_books = None
limit_to_circulation = False
for index, row in partial_df.iterrows():
    print(index)
    print(row.subscription_start, row.subscription_end)
    circulation_events = events_df[(events_df.start_datetime < row.subscription_endtime) | (events_df.end_datetime < row.subscription_endtime)]
    print('identified events', len(circulation_events))
    query_books = circulation_events[circulation_events.item_uri.notna()].item_uri.unique().tolist()
    print('identified books', len(query_books))
    member_book_ids = events_df[(events_df.item_uri.notna()) & (events_df.member_id == row.member_id)].item_uri.unique()
    query_books = list(set(query_books) - set(member_book_ids))
    print('identified books without those read by member', len(query_books))
    if limit_to_circulation:
        if index == 0:
            previous_books = query_books
        else:
            query_books = list(set(query_books) - set(previous_books))
            print('identified books without those read by member and those read in previous subscription', len(query_books))
    rankings, cos_rankings, pear_rankings = get_predictions(row.member_id, d, weighted, query_books)
    print('euclidean:',len(rankings), 'cosine:', len(cos_rankings), 'pearson:',len(pear_rankings))


    euc_df = pd.DataFrame(rankings[0:n])
    euc_df.columns = ['score', 'item_uri']
    chart_titles = get_formatted_chart_titles(euc_df, n)
    euc_df['formatted_chart_title'] = chart_titles
    table_titles = get_formatted_table_titles(euc_df, n)
    euc_df['formatted_table_title'] = table_titles
    euc_df['type'] = 'euclidean'

    cos_df = pd.DataFrame(cos_rankings[0:n])
    cos_df.columns = ['score', 'item_uri']
    chart_titles = get_formatted_chart_titles(cos_df, n)
    cos_df['formatted_chart_title'] = chart_titles
    table_titles = get_formatted_table_titles(cos_df, n)
    cos_df['formatted_table_title'] = table_titles
    cos_df['type'] = 'cosine'

    pear_df = pd.DataFrame(pear_rankings[0:n])
    pear_df.columns = ['score', 'item_uri']
    chart_titles = get_formatted_chart_titles(pear_df, n)
    pear_df['formatted_chart_title'] = chart_titles
    table_titles = get_formatted_table_titles(pear_df, n)
    pear_df['formatted_table_title'] = table_titles
    pear_df['type'] = 'pearson'

    dfs = pd.concat([euc_df, cos_df, pear_df])
    dfs['member_id'] = row.member_id
    dfs['period'] = row.subscription_start + '/' + row.subscription_end
    dfs['subscription_start'] = row.subscription_start
    dfs['subscription_end'] = row.subscription_end
    # print(len(dfs))
    final_df.append(dfs)


25
1921-12-28 1922-11-08
identified events 2642
identified books 666
identified books without those read by member 660
euclidean: 551 cosine: 551 pearson: 551
26
1924-03-28 1925-03-28
identified events 6241
identified books 1310
identified books without those read by member 1298
euclidean: 1108 cosine: 1108 pearson: 1108


In [193]:
final_preds = pd.concat(final_df)
pivoted_predictions =pd.pivot(final_preds, index=['item_uri', 'formatted_chart_title', 'formatted_table_title', 'member_id', 'period', 'subscription_start', 'subscription_end'], columns='type', values='score').reset_index().fillna(0)

In [160]:
len(final_preds)

1200

In [161]:
final_preds['score_scaled'] = final_preds.groupby(['period', 'type'])['score'].apply(lambda x: (x-min(x))/(max(x)-min(x)))

In [194]:
def scale_col(df, cols):
    for col in cols:
        df[col + '_scaled'] = MinMaxScaler().fit_transform(df[col].values.reshape(-1, 1))
    return df
metrics = ['cosine', 'pearson', 'euclidean']
scaled_predictions = scale_col(pivoted_predictions, metrics)
preds_df = pd.melt(scaled_predictions, id_vars=['item_uri', 'formatted_chart_title', 'formatted_table_title', 'member_id', 'period', 'subscription_start', 'subscription_end'], value_vars=['cosine_scaled', 'pearson_scaled', 'euclidean_scaled'], var_name='metric', value_name='score')

In [195]:
preds_df.loc[preds_df.metric == 'cosine_scaled', 'metric'] = 'cosine'
preds_df.loc[preds_df.metric == 'pearson_scaled', 'metric'] = 'pearson'
preds_df.loc[preds_df.metric == 'euclidean_scaled', 'metric'] = 'euclidean'

In [196]:
preds_df = preds_df.sort_values(by=['score'] ,ascending=False)
items = preds_df[0:100].item_uri.unique()
print(len(items))

62


In [197]:
periods = preds_df.period.unique().tolist()
n =23
final_items = []
for index, period in enumerate(periods):
    if index == 0:
        items = []
        rows = preds_df[(preds_df.period == period) & (preds_df.formatted_chart_title.str.contains('Periodical') == False)].sort_values(by=['score'] ,ascending=False)
        while len(items) <= n:
            item = rows.item_uri.iloc[0]
            if item not in items:
                items.append(item)
            rows = rows.drop(rows.index[0])
        final_items.append(items)
    else:
        items = []
        rows = preds_df[(preds_df.period == period) & (preds_df.formatted_chart_title.str.contains('Periodical') == False)].sort_values(by=['score'] ,ascending=False)
        while len(items) <= n:
            item = rows.item_uri.iloc[0]
            if (item not in items) and (item not in final_items[index-1]):
                items.append(item)
            rows = rows.drop(rows.index[0])
        items = items[0:n]
        # print(len(items))
        final_items.append(items)

In [198]:
len(final_items[1])

23

In [199]:
top_items = final_items[0] + final_items[1] 

In [200]:
preds_df[preds_df.item_uri.isin(top_items)].groupby(['period'])['item_uri'].nunique()

period
1921-12-28/1922-11-08    30
1924-03-28/1925-03-28    47
Name: item_uri, dtype: int64

In [201]:
top_results = preds_df[preds_df.item_uri.isin(top_items)]
top_results.to_csv('./public_data/memorycf_top_results.csv', index=False)

In [99]:
preds_df[(preds_df.formatted_chart_title.str.contains('Trollope')) & (preds_df.item_uri.isin(top_items))]

,item_uri,formatted_chart_title,formatted_table_title,member_id,period,subscription_start,subscription_end,metric,score


In [174]:
chart = alt.Chart(preds_df[preds_df.item_uri.isin(top_items)]).mark_point(opacity=0.8, filled=True, thickness=3).encode(
    y=alt.Y('formatted_chart_title', sort='-x',  axis=alt.Axis(title='Predicted Book', orient='right')),
    x='score',
    color=alt.Color('period:N', legend=alt.Legend(title='Subscription Periods With Missing Records', orient='left')),
    # column='member_id:N',
    shape=alt.Shape('metric:N'),
).properties(
    width=300,
    title='Top Predictions by Collaborative Filtering Memory-Based Methods'
)
chart = chart.configure_axisY(
        titleAngle=0,
        titleAlign="left",
        titleY=-10,
        titleX=-100,
        labelLimit=1000
    )

In [175]:
chart

alt.Chart(...)

In [141]:
# preds_df = final_preds.copy()
# preds_df['score'] = preds_df.score_scaled

In [176]:
import numpy as np
metrics_df = preds_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title']).agg({'score': [np.median, 'skew', 'std', 'var']}).reset_index()
metrics_df.columns = list(map(''.join, metrics_df.columns.values))
metrics_df.columns = [col if 'score' not in col else col.split('score')[1] for col in metrics_df.columns ]
kurt_df = preds_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])['score'].apply(pd.DataFrame.kurt).reset_index(name='kurtosis')
final_df = pd.merge(metrics_df, kurt_df, on=['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])
final_df = pd.merge(final_df, preds_df, on=['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'], how='left')

In [177]:
from scipy.stats import zscore, mode
final_df['zscore'] = final_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])['score'].transform(lambda x : zscore(x,ddof=1))

In [178]:
top_scores = final_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title']).agg({'score':'max'})[['score']].reset_index()
top_scores = pd.merge(top_scores, final_df, on=top_scores.columns.tolist(), how='inner')

top_scores = top_scores.rename(columns={'score': 'top_score', 'zscore' : 'top_zscore'})

In [179]:
avg_scores = final_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])['score'].mean().reset_index(name='avg_score')
scores_df = pd.merge(top_scores, avg_scores, on=['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])

In [180]:
median_scores = final_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])['score'].median().reset_index(name='median_score')
scores_df = pd.merge(scores_df, median_scores, on=['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])

In [181]:
std_scores = final_df.groupby(['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title']).agg({'score': 'std'}).reset_index()
std_scores = std_scores.rename(columns={'score': 'std_score'})
scores_df = pd.merge(scores_df, std_scores, on=['member_id', 'subscription_start', 'subscription_end', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title'])

In [182]:
mode_scores = preds_df.groupby(['member_id', 'period', 'subscription_start', 'subscription_end','item_uri'])['score'].agg(lambda x: pd.Series.mode(x).iat[0]).reset_index()
mode_scores = pd.merge(mode_scores, final_df, on=mode_scores.columns.tolist(), how='inner')
mode_scores = mode_scores.rename(columns={'score': 'mode_score', 'zscore' : 'mode_zscore'})

In [183]:
final_scores = pd.merge(mode_scores[['member_id', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title', 'mode_score', 'mode_zscore', 'subscription_start', 'subscription_end']], scores_df, on=['member_id', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title', 'subscription_start', 'subscription_end'])

In [184]:
final_scores

,member_id,period,item_uri,formatted_table_title,formatted_chart_title,mode_score,mode_zscore,subscription_start,subscription_end,top_score,median,skew,std,var,kurtosis,metric,top_zscore,avg_score,median_score,std_score
0,hemingway-ernest,1921-12-28/1922-11-08,a-e-imaginations-reveries,"*Imaginations and Reveries*,<br>Æ",Imaginations and Reveries byÆ,0.024169,-1.111490,1921-12-28,1922-11-08,0.221125,0.166048,-1.177452,0.101616,0.010326,NaN,euclidean,0.826748,0.137114,0.166048,0.101616
1,hemingway-ernest,1921-12-28/1922-11-08,abercrombie-thomas-hardy-critical,"*Thomas Hardy: A Critical Study*,<br> Lascelle...",Thomas Hardy: A Critical Study by Lascelles Ab...,0.039540,-1.075610,1921-12-28,1922-11-08,0.428571,0.285434,-0.759614,0.196765,0.038716,NaN,euclidean,0.901532,0.251182,0.285434,0.196765
2,hemingway-ernest,1921-12-28/1922-11-08,barrie-kiss-cinderella,"*A Kiss for Cinderella: A Comedy*,<br> J. M. B...",A Kiss for Cinderella: A Comedy by J. M. Barrie,0.000000,-0.577350,1921-12-28,1922-11-08,0.019770,0.000000,1.732051,0.011414,0.000130,NaN,pearson,1.154701,0.006590,0.000000,0.011414
3,hemingway-ernest,1921-12-28/1922-11-08,barrie-kiss-cinderella,"*A Kiss for Cinderella: A Comedy*,<br> J. M. B...",A Kiss for Cinderella: A Comedy by J. M. Barrie,0.000000,-0.577350,1921-12-28,1922-11-08,0.019770,0.000000,1.732051,0.011414,0.000130,NaN,pearson,1.154701,0.006590,0.000000,0.011414
4,hemingway-ernest,1921-12-28/1922-11-08,beerbohm-zuleika-dobson,"*Zuleika Dobson*,<br> Max Beerbohm",Zuleika Dobson by Max Beerbohm,0.026132,-1.097906,1921-12-28,1922-11-08,0.226531,0.163079,-1.014785,0.102421,0.010490,NaN,euclidean,0.858713,0.138580,0.163079,0.102421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,hemingway-ernest,1924-03-28/1925-03-28,wright-life-walter-pater,"*The Life of Walter Pater*,<br> Thomas Wright",The Life of Walter Pater by Thomas Wright,0.039540,-1.075610,1924-03-28,1925-03-28,0.428571,0.285434,-0.759614,0.196765,0.038716,NaN,euclidean,0.901532,0.251182,0.285434,0.196765
513,hemingway-ernest,1924-03-28/1925-03-28,yeats-plays,"*Plays*,<br> William Butler Yeats",Plays by William Butler Yeats,0.026220,-1.111089,1924-03-28,1925-03-28,0.220779,0.166144,-1.172574,0.100347,0.010069,NaN,euclidean,0.827777,0.137715,0.166144,0.100347
514,hemingway-ernest,1924-03-28/1925-03-28,yeats-plays-controversies,"*Plays and Controversies*,<br> William Butler ...",Plays and Controversies by William Butler Yeats,0.031707,-1.110428,1924-03-28,1925-03-28,0.285714,0.213894,-1.164532,0.130939,0.017145,NaN,euclidean,0.829463,0.177105,0.213894,0.130939
515,hemingway-ernest,1924-03-28/1925-03-28,yeats-responsibilities-poems,"*Responsibilities and Other Poems*,<br> Willia...",Responsibilities and Other Poems by William Bu...,0.034166,-1.131807,1924-03-28,1925-03-28,0.230769,0.189674,-1.431100,0.103702,0.010754,NaN,euclidean,0.764045,0.151536,0.189674,0.103702


In [185]:
member_subscriptions = final_scores[['member_id', 'period', 'item_uri', 'formatted_table_title', 'formatted_chart_title', 'subscription_start', 'subscription_end']].drop_duplicates()

In [186]:
final_scores_dedup = final_scores.drop_duplicates()

In [187]:
final_scores_dedup['coef_var'] = (final_scores_dedup.std_score/ final_scores_dedup.median_score)

In [188]:
# for index, group in member_subscriptions.iterrows():
#   print(group.to_dict())
#   rows = final_scores_dedup[(final_scores_dedup.member_id == group.member_id) & (final_scores_dedup.period == group.period) ]
#   print('coef_scores:', rows.sort_values(by=['coef_var'], ascending=True)[0:5][['item_uri', 'coef_var', 'median_score']].to_dict())
#   print('median_scores:', rows.sort_values(by=['median_score'], ascending=False)[0:5][['item_uri', 'median_score', 'coef_var']].to_dict())

In [189]:
len(final_scores), len(final_scores_dedup)

(517, 460)

In [190]:
for period in final_scores_dedup.period.unique().tolist():

    rows = final_scores_dedup[final_scores_dedup.period == period].sort_values(by=['median_score', 'coef_var'], ascending=[False, True])
    print(rows[['formatted_chart_title']].head(10))

                                 formatted_chart_title
94                             The Nation (Periodical)
143         The Times Literary Supplement (Periodical)
173                    The London Mercury (Periodical)
125            Specimens of Early English (Periodical)
1    Thomas Hardy: A Critical Study by Lascelles Ab...
5                          Since Cézanne by Clive Bell
78                                Howells (Periodical)
84                          The Clash by Storm Jameson
87                Samuel Butler by Henry Festing Jones
101  The Daniel Jazz and Other Poems by Vachel Lindsay
                             formatted_chart_title
362                        The Nation (Periodical)
485  Works of Francis Thompson by Francis Thompson
438     The Times Literary Supplement (Periodical)
392              The American Mercury (Periodical)
286          The Theory of Beauty by E. F. Carritt
296                     The Criterion (Periodical)
459                The London Mercury 

In [75]:
final_scores_dedup[final_scores_dedup.formatted_chart_title.str.contains('Trollope')]

,member_id,period,item_uri,formatted_table_title,formatted_chart_title,mode_score,mode_zscore,subscription_start,subscription_end,top_score,median,skew,std,var,kurtosis,metric,top_zscore,avg_score,median_score,std_score,coef_var
488,hemingway-ernest,1924-03-28/1925-03-28,trollope-bertrams,"*The Bertrams*,<br> Anthony Trollope",The Bertrams by Anthony Trollope,0.035482,-1.154517,1924-03-28,1925-03-28,0.241071,0.236885,-1.729578,0.117508,0.013808,NaN,euclidean,0.595070,0.171146,0.236885,0.117508,0.496052
489,hemingway-ernest,1924-03-28/1925-03-28,trollope-claverings,"*The Claverings*,<br> Anthony Trollope",The Claverings by Anthony Trollope,0.026898,-1.126980,1924-03-28,1925-03-28,0.222527,0.177871,-1.369710,0.102517,0.010510,NaN,euclidean,0.781290,0.142432,0.177871,0.102517,0.576352
490,hemingway-ernest,1924-03-28/1925-03-28,trollope-last-chronicle-barset,"*The Last Chronicle of Barset*,<br> Anthony Tr...",The Last Chronicle of Barset by Anthony Trollope,0.080533,-1.104105,1924-03-28,1925-03-28,0.355270,0.272727,-1.088332,0.140968,0.019872,NaN,cosine,0.844822,0.236177,0.272727,0.140968,0.516883
491,hemingway-ernest,1924-03-28/1925-03-28,trollope-prime-minister,"*The Prime Minister*,<br> Anthony Trollope",The Prime Minister by Anthony Trollope,0.027050,-1.128214,1924-03-28,1925-03-28,0.221198,0.177798,-1.385334,0.101900,0.010384,NaN,euclidean,0.777062,0.142015,0.177798,0.101900,0.573126
492,hemingway-ernest,1924-03-28/1925-03-28,trollope-small-house-allington,"*The Small House at Allington*,<br> Anthony Tr...",The Small House at Allington by Anthony Trollope,0.052491,-1.154649,1924-03-28,1925-03-28,0.236686,0.234694,-1.731360,0.105775,0.011188,NaN,cosine,0.586741,0.174624,0.234694,0.105775,0.450692


In [158]:
final_scores_dedup.to_csv('./public_data/collaborative_filtering_predictions.csv', index=False)

In [60]:
final_scores

,member_id,period,item_uri,formatted_table_title,formatted_chart_title,mode_score,mode_zscore,top_score,median,skew,std,var,kurtosis,metric,top_zscore,avg_score,median_score,std_score
0,hemingway-ernest,1921-12-28/1922-11-08,a-e-imaginations-reveries,"*Imaginations and Reveries*,<br>Æ",Imaginations and Reveries byÆ,0.024169,-1.111490,0.221125,0.166048,-1.177452,0.101616,0.010326,NaN,euclidean,0.826748,0.137114,0.166048,0.101616
1,hemingway-ernest,1921-12-28/1922-11-08,abercrombie-thomas-hardy-critical,"*Thomas Hardy: A Critical Study*,<br> Lascelle...",Thomas Hardy: A Critical Study by Lascelles Ab...,0.039540,-1.075610,0.428571,0.285434,-0.759614,0.196765,0.038716,NaN,euclidean,0.901532,0.251182,0.285434,0.196765
2,hemingway-ernest,1921-12-28/1922-11-08,barrie-kiss-cinderella,"*A Kiss for Cinderella: A Comedy*,<br> J. M. B...",A Kiss for Cinderella: A Comedy by J. M. Barrie,0.000000,-0.577350,0.019770,0.000000,1.732051,0.011414,0.000130,NaN,pearson,1.154701,0.006590,0.000000,0.011414
3,hemingway-ernest,1921-12-28/1922-11-08,barrie-kiss-cinderella,"*A Kiss for Cinderella: A Comedy*,<br> J. M. B...",A Kiss for Cinderella: A Comedy by J. M. Barrie,0.000000,-0.577350,0.019770,0.000000,1.732051,0.011414,0.000130,NaN,pearson,1.154701,0.006590,0.000000,0.011414
4,hemingway-ernest,1921-12-28/1922-11-08,beerbohm-zuleika-dobson,"*Zuleika Dobson*,<br> Max Beerbohm",Zuleika Dobson by Max Beerbohm,0.026132,-1.097906,0.226531,0.163079,-1.014785,0.102421,0.010490,NaN,euclidean,0.858713,0.138580,0.163079,0.102421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,hemingway-ernest,1924-03-28/1925-03-28,woolf-jacobs-room,"*Jacob's Room*,<br> Virginia Woolf",Jacob's Room by Virginia Woolf,0.021104,-1.067627,0.234127,0.152253,-0.671771,0.107457,0.011547,NaN,euclidean,0.914773,0.135828,0.152253,0.107457
513,hemingway-ernest,1924-03-28/1925-03-28,woolf-night-day,"*Night and Day*,<br> Virginia Woolf",Night and Day by Virginia Woolf,0.023264,-1.101864,0.217897,0.158259,-1.061608,0.099718,0.009944,NaN,euclidean,0.849967,0.133140,0.158259,0.099718
514,hemingway-ernest,1924-03-28/1925-03-28,wright-life-walter-pater,"*The Life of Walter Pater*,<br> Thomas Wright",The Life of Walter Pater by Thomas Wright,0.039540,-1.075610,0.428571,0.285434,-0.759614,0.196765,0.038716,NaN,euclidean,0.901532,0.251182,0.285434,0.196765
515,hemingway-ernest,1924-03-28/1925-03-28,yeats-plays,"*Plays*,<br> William Butler Yeats",Plays by William Butler Yeats,0.026220,-1.111089,0.220779,0.166144,-1.172574,0.100347,0.010069,NaN,euclidean,0.827777,0.137715,0.166144,0.100347


In [17]:
cosine_sim = linear_kernel(weighted, weighted)
members = weighted.copy()
members['member_id'] = members.index
members = members[[ 'member_id']]
members.reset_index(drop=True, inplace=True)


In [19]:
def get_cosine_recommendations(member_id: str, cosine_sim=cosine_sim, members=members, numb_of_members=10):
    # Get the index of the member
    member_index = members[members['member_id'] == member_id].index[0]

    # Get the pairwsie similarity scores of all members for the given member
    sim_scores = list(enumerate(cosine_sim[member_index]))

    # Sort the scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar members
    sim_scores = sim_scores[1:numb_of_members+1]

    # Get the member indices
    member_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar members
    return members.iloc[member_indices]



In [23]:


# Checks for similarity to Maureen using Euclidean distance score
def top_matches(prefs, person, n=10, similarity = euc_distance):
    scores = [(similarity(prefs,person,other), other)
            for other in prefs if other!=person]
    scores.sort()
    scores.reverse()
    return scores[0:n]


def get_recommendations(prefs, person, n=10, similarity = euc_distance):
    totals = {} 
    simSums = {}
    for other in prefs:
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        if sim <= 0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item,0) 
                totals[item] += prefs[other][item] * sim
                simSums.setdefault(item,0)
                simSums[item] += sim
    rankings = [(total/simSums[item], item) for item, total in totals.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[0:n]

In [31]:
cosine_sim = linear_kernel(weighted, weighted)
members = weighted.copy()
members['member_id'] = members.index
members = members[[ 'member_id']]
members.reset_index(drop=True, inplace=True)

In [72]:
hem_test = weighted[weighted.index == 'hemingway-ernest'].values.tolist()[0]
alex_test = weighted[weighted.index == 'arvanon-cyrille'].values.tolist()[0]

0.9978093856514196

In [77]:
member = 'hemingway-ernest'
df = weighted.copy()

In [86]:
member_vector = df.loc[member]
similarities = df.drop(member).apply(lambda x: pearsonr(x, member_vector), axis=1)

In [88]:
test2 = pd.DataFrame(similarities).reset_index()
test2.columns = ['member_id', 'similarity']
test2[test2.member_id == 'arvanon-cyrille']

,member_id,similarity
5,arvanon-cyrille,"(-0.0013965614111280005, 0.9164733531251589)"


In [70]:
member_id = 'hemingway-ernest'
other_member = 'arvanon-cyrille'
member_index = members[members['member_id'] == member_id].index[0]
other_index = members[members['member_id'] == other_member].index[0]

# Get the pairwise similarity scores of all members for the given member
sim_scores = list(enumerate(cosine_sim[member_index]))

# Sort the scores in descending order
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)


# # Get the scores of the 10 most similar members
# sim_scores = sim_scores[1:numb_of_members+1]

# # Get the member indices
# member_indices = [i[0] for i in sim_scores]


In [71]:
test = pd.DataFrame(sim_scores, columns=['member_index', 'score'])
test[test.member_index == other_index]

,member_index,score
50,5,34.756156


In [105]:
pd.DataFrame([totals])

anderson-kit-brandon-portrait  anderson-many-marriages  \
0                       5.602817                      6.2   

   anderson-marching-men  anderson-poor-white  anderson-story-tellers-story  \
0                    3.0                  5.5                           3.2   

   anderson-triumph-egg  baudelaire-flowers-evil  \
0                   5.5                 1.002817   

   beach-twentieth-century-novel  benet-john-browns-body  \
0                            5.5                     1.0   

   boyle-first-lover-stories  browne-vulgar-errors-pseudodoxia  \
0                        3.7                               2.0   

   calmer-proletarian-literature-united  cather-song-lark  \
0                              2.502817               4.0   

   crane-maggie-girl-streets  cummings-tulips-chimneys  \
0                        5.5                       1.5   

   day-lewis-friendly-tree  de-quincey-reminiscences-english-lake  \
0                 4.102817                                    3.0   

   dickinson-poems-emily-dickinson  dos-passos-42nd-parallel  \
0                              2.5                       7.5   

   dos-passos-big-money  dos-passos-three-plays  dos-passos-three-soldiers  \
0              7.102817                     2.0                        8.6   

   dreiser-free-stories  dreiser-twelve-men  early-life-thomas  \
0                   3.0                 3.5                1.0   

   eliot-family-reunion  eliot-murder-cathedral  empson-seven-types-ambiguity  \
0                   7.0                8.333333                      1.333333   

   farrell-studs-lonigan  fausset-samuel-taylor-coleridge  \
0                    2.5                              5.0   

   forster-howards-end  hanley-secret-journey  hart-crane  \
0                 16.7                    2.1         2.5   

   hertz-hidden-lincoln  hudson-side  isherwood-mr-norris-changes  \
0                   2.5     6.833333                    17.969484   

   lehmann-dusty-answer  lehmann-note-music  lehmann-weather-streets  \
0                   8.1           10.333333                10.233333   

   lewis-prodigal-parents  lindsay-collected-poems  macleish-poems-1924-1933  \
0                2.102817                      2.0                       2.5   

   melville-moby-dick-whale  moore-selected-poems-marianne  \
0                      14.2                            4.0   

   moore-selected-poems-thomas  mumford-culture-cities  new-testament  \
0                          1.5                     1.0            1.0   

   pattee-new-american-literature  pound-selected-poems  \
0                             1.0                   2.0   

   sackville-west-passion-spent  sitwell-collected-poems-edith  spender-poems  \
0                           3.1                            4.0            5.0   

   stevens-man-blue-guitar  wilde-english-renaissance-lecture  \
0                      1.0                                1.6   

   wilder-heavens-destination  williams-white-mule  \
0                    8.669484                  3.6   

   wylie-collected-poems-elinor  addison-spectator  auden-dog-beneath-skin  \
0                      3.002817                0.5                     7.0   

   bagnold-squire  boswell-life-samuel-johnson  bottome-murder-bud  \
0             2.5                          7.2            6.433333   

   bowen-death-heart  bowen-faber-book-modern  buck-patriot  \
0                9.1                 3.666667           5.1   

   bunyan-pilgrims-progress  burney-story-fanny-burney  \
0                       1.0                        2.5   

   calder-marshall-dead-centre  canby-house  carlyle-past-present  \
0                          7.5          0.5                   1.5   

   christian-symbols  connolly-enemies-promise  \
0                0.5                  6.666667   

   coomaraswamy-buddha-gospel-buddhism  dos-passos-adventures-young-man  \
0                                  0.5                     

In [140]:
borrow_events[borrow_events.item_uri == 'wells-outline-history']

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,subscription_volumes,subscription_category,subscription_purchase_date,reimbursement_refund,borrow_status,borrow_duration_days,purchase_price,currency,item_uri,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image,first_member_uri,second_member_uri,member_id,index_col,dt,exceptional_types,books_out,excess_books_out,start_datetime,end_datetime,subscription_purchase_datetime,index,year,month
1081,Borrow,1920-11-13,1920-11,https://shakespeareandco.princeton.edu/members...,Madeleine Rolland,"Rolland, Madeleine",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,NaN,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, Madeleine Rolland Lending Librar...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/d5%2F8...,https://shakespeareandco.princeton.edu/members...,NaN,rolland-madeleine,1088,1920-11-13,NaN,NaN,NaN,1920-11-13,1920-11-01,NaT,1081,1920.0,11.0
1151,Borrow,1920-12-13,1920-12-28,https://shakespeareandco.princeton.edu/members...,Madeleine Rolland,"Rolland, Madeleine",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,15.0,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, Madeleine Rolland Lending Librar...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif-cloud.princeton.edu/iiif/2/d5%2F8...,https://shakespeareandco.princeton.edu/members...,NaN,rolland-madeleine,1158,1920-12-13,NaN,NaN,NaN,1920-12-13,1920-12-28,NaT,1151,1920.0,12.0
1621,Borrow,1921-10-06,1921-10-28,https://shakespeareandco.princeton.edu/members...,John Varney,"Varney, John",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,22.0,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, John Varney Lending Library Card...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/e8...,https://shakespeareandco.princeton.edu/members...,NaN,varney,1633,1921-10-06,NaN,NaN,NaN,1921-10-06,1921-10-28,NaT,1621,1921.0,10.0
1919,Borrow,1922-02-02,1922-02-09,https://shakespeareandco.princeton.edu/members...,Maud Martin,"Martin, Maud",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,7.0,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, Maud Martin Lending Library Card...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/20...,https://shakespeareandco.princeton.edu/members...,NaN,martin-maud,1934,1922-02-02,NaN,NaN,NaN,1922-02-02,1922-02-09,NaT,1919,1922.0,2.0
3198,Borrow,1923-08-27,1923-09-03,https://shakespeareandco.princeton.edu/members...,Thelma Wood,"Wood, Thelma",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,7.0,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, Miss Thelma Wood Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/d5...,https://shakespeareandco.princeton.edu/members...,NaN,wood-thelma,3231,1923-08-27,NaN,NaN,NaN,1923-08-27,1923-09-03,NaT,3198,1923.0,8.0
3256,Borrow,1923-09-17,1923-10-11,https://shakespeareandco.princeton.edu/members...,Thelma Wood,"Wood, Thelma",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Returned,24.0,NaN,NaN,wells-outline-history,The Outline of History,NaN,"Wells, H. G.",1920.0,NaN,Lending Library Card,"Sylvia Beach, Miss Thelma Wood Lending Library...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/5c...,https://shakespeareandco.princeton.edu/members...,NaN,wood-thelma,3289,1923-09-17,NaN,NaN,NaN,1923-09-17,1923-10-11,NaT,3256,1923.0,9.0
3895,Borrow,1924-04-04,1924-04-08,https://shakespeareandc

In [139]:
top_df = pd.concat([euc_df, cos_df, pear_df])
# top_df[top_df.formatted_title.str.contains('Periodical') == False]
top_df

,score,item_uri,formatted_title,type
0,10.994371,ross-new-yorker,"*The New Yorker*,<br>(Periodical)",euclidean
1,10.000000,ward-cambridge-history-english,"*The Cambridge History of English Literature*,...",euclidean
2,7.000000,hartman-harpers-magazine,"*Harper's Magazine*,<br>(Periodical)",euclidean
3,7.000000,anglo-saxon-dictionary,"*Anglo-Saxon Dictionary*,<br>(Periodical)",euclidean
4,6.000000,national-geographic,"*National Geographic*,<br>(Periodical)",euclidean
5,4.666667,thompson-francis-thompsons-works,"*Works of Francis Thompson*,<br> Francis Thompson",euclidean
6,4.600000,richmond-times-literary-supplement,"*The Times Literary Supplement*,<br>(Periodical)",euclidean
7,4.000000,walpole-fortitude,"*Fortitude*,<br> Hugh Walpole",euclidean
8,4.000000,gordon-anglo-saxon-poetry,"*Anglo-Saxon Poetry*,<br>(Periodical)",euclidean
9,3.852141,maccarthy-life-letters,"*Life and Letters*,<br>(Periodical)",euclidean


In [21]:
get_recommendations(d, 'hemingway-ernest')

[(10.99437148217636, 'ross-new-yorker'),
 (10.0, 'ward-cambridge-history-english'),
 (7.0, 'hartman-harpers-magazine'),
 (7.0, 'anglo-saxon-dictionary'),
 (6.0, 'national-geographic'),
 (4.666666666666667, 'thompson-francis-thompsons-works'),
 (4.6, 'richmond-times-literary-supplement'),
 (4.0, 'walpole-fortitude'),
 (4.0, 'gordon-anglo-saxon-poetry'),
 (3.8521412250313256, 'maccarthy-life-letters')]

In [22]:
'ross-new-yorker' in member_book_ids

NameError: name 'member_book_ids' is not defined

In [1]:
# from scipy import sparse

# # create scipy sparse from pivot tables
# data_sparse = sparse.csr_matrix(df)
# similarities_sparse = cosine_similarity(data_sparse, dense_output=False)
# # returns index (column position) of top n similarities in each row
# import numpy as np
# def top_n_idx_sparse(matrix, n):
#     '''Return index of top n values in each row of a sparse matrix'''
#     top_n_idx = []
#     for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
#         n_row_pick = min(n, ri - le)
#         top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
#     return top_n_idx

# user_user_similar = top_n_idx_sparse(similarities_sparse, 15)

# user_user_similar_dict = {}
# for idx, val in enumerate(user_user_similar):
#     user_user_similar_dict.update({idx: val.tolist()})

# similar_users_final = {}
# for user, similar_users in user_user_similar_dict.items():
#     idx = df.index[user]
#     values = []
#     for value in similar_users:
#         values.append(df.index[value])

#     similar_users_final.update({idx: values})

# heming_sim = similar_users_final['hemingway-ernest']

# grouped_borrows[(grouped_borrows['member_id'].isin(heming_sim))].sort_values(by='counts', ascending=False)
# # .sample(n=10, random_state=33)['item_uri'].values

# # user_movies.update({user: list(set(movies_sample))})

# user_movies = {}
# for user, similar_users in similar_users_final.items():
#   if user == 'hemingway-ernest':
#     # remove the user itself from similar_users (since cos_sim(user_1, user_1) is 1)
#     try:
#         del similar_users[similar_users.index(user)]
#     except:
#         pass
#     # get movie ids from list of movies rated by similar users.
#     # also apply extra logic to get the most high rated movies from the similar users
#     movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=3]
#     # if movies_rec.empty:
#     #   movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=2]
#     # if movies_rec.empty:
#     #   movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=1]
#     movies_sample = movies_rec.sample(n=10, random_state=33)['item_uri'].values
#     user_movies.update({user: list(set(movies_sample))})

# user_movies

In [ ]:
# from scipy import sparse

# # create scipy sparse from pivot tables
# data_sparse = sparse.csr_matrix(df)
# similarities_sparse = cosine_similarity(data_sparse, dense_output=False)
# # returns index (column position) of top n similarities in each row
# import numpy as np
# def top_n_idx_sparse(matrix, n):
#     '''Return index of top n values in each row of a sparse matrix'''
#     top_n_idx = []
#     for le, ri in zip(matrix.indptr[:-1], matrix.indptr[1:]):
#         n_row_pick = min(n, ri - le)
#         top_n_idx.append(matrix.indices[le + np.argpartition(matrix.data[le:ri], -n_row_pick)[-n_row_pick:]])
#     return top_n_idx

# user_user_similar = top_n_idx_sparse(similarities_sparse, 15)

# user_user_similar_dict = {}
# for idx, val in enumerate(user_user_similar):
#     user_user_similar_dict.update({idx: val.tolist()})

# similar_users_final = {}
# for user, similar_users in user_user_similar_dict.items():
#     idx = df.index[user]
#     values = []
#     for value in similar_users:
#         values.append(df.index[value])

#     similar_users_final.update({idx: values})

# heming_sim = similar_users_final['hemingway-ernest']

# grouped_borrows[(grouped_borrows['member_id'].isin(heming_sim))].sort_values(by='counts', ascending=False)
# # .sample(n=10, random_state=33)['item_uri'].values

# # user_movies.update({user: list(set(movies_sample))})

# user_movies = {}
# for user, similar_users in similar_users_final.items():
#   if user == 'hemingway-ernest':
#     # remove the user itself from similar_users (since cos_sim(user_1, user_1) is 1)
#     try:
#         del similar_users[similar_users.index(user)]
#     except:
#         pass
#     # get movie ids from list of movies rated by similar users.
#     # also apply extra logic to get the most high rated movies from the similar users
#     movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=3]
#     # if movies_rec.empty:
#     #   movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=2]
#     # if movies_rec.empty:
#     #   movies_rec = grouped_borrows[(grouped_borrows['member_id'].isin(similar_users)) & grouped_borrows['counts']>=1]
#     movies_sample = movies_rec.sample(n=10, random_state=33)['item_uri'].values
#     user_movies.update({user: list(set(movies_sample))})

# user_movies

In [ ]:

# def member_query(df,member: str,num_books: int = 5,num_members: int = 5):
#     print(member)
#     print('Top Book Scores:')
#     # print(df.loc[member].sort_values(ascending=False)[:num_books])
#     updated_similarities = get_formatted_titles(df.loc[member].sort_values(ascending=False).reset_index(name='score'), num_books)
#     print(updated_similarities)

#     member_vector = df.loc[member]
#     similarities = df.drop(member).apply(
#         lambda x: pearsonr(x, member_vector), axis=1)

#     print('Most Similar members:')
#     print(similarities.sort_values()[::-1][:num_members])
#     print()

In [ ]:
# from scipy.stats import pearsonr
# num_books = 10
# dfs = []
# initial_scores = None
# query_books = None
# for index, row in partial_df.iterrows():
#         # circulation_events = borrow_df[borrow_df.start_datetime.between(relative_date, row.subscription_endtime) | borrow_df.end_datetime.between(relative_date, row.subscription_endtime)]
#         # exclude any that this user is known to have borrowed 
#         circulation_events = borrow_events[(borrow_events.start_datetime < row.subscription_end) | (borrow_events.end_datetime < row.subscription_end)]
        
#         query_books = circulation_events.item_uri.unique().tolist()
#         member_book_ids = borrow_events[(borrow_events.item_uri.notna()) & (borrow_events.member_id == row.member_id)].item_uri.unique()
#         query_books = list(set(query_books) - set(member_book_ids))
#         scores = weighted.loc[row.member_id].sort_values(ascending=False).reset_index(name='score')
#         initial_scores = scores
#         scores = scores[scores.item_uri.isin(query_books)]
#         updated_titles = get_formatted_titles(scores, num_books)
#         final_scores = scores[:num_books]
#         final_scores['formatted_titles'] = updated_titles
#         final_scores['period'] = row.subscription_start + '/' + row.subscription_end
#         dfs.append(final_scores)


In [ ]:
member_query(weighted, 'hemingway-ernest')
# member_dict = {}
# feat_cols = weighted.columns.tolist()
# feat_cols.remove('author')
# for member in weighted.index:
#     row = weighted.loc[weighted.author == member].copy()
#     top_10 = row.melt(id_vars='author', value_vars=feat_cols).sort_values(by='value', ascending=False).head(10)
#     member_dict[member] = top_10[['item_uri', 'value']].to_dict('records')
